In [20]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt

In [21]:
torch.manual_seed(42)
transforms= T.Compose([T.ToTensor(),T.Normalize([0.5],[0.5])])

In [42]:
train_set=torchvision.datasets.FashionMNIST(root=r'C:\Users\alibr\OneDrive\Desktop\pytorch' ,download=True,train=True,transform=transforms )
test_set=torchvision.datasets.FashionMNIST(root=r'C:\Users\alibr\OneDrive\Desktop\pytorch' ,download=True,train=False,transform=transforms)
train_set,val_set=torch.utils.data.random_split(train_set,[50000,10000])

In [43]:
train_loader=torch.utils.data.DataLoader(train_set,batch_size=64,shuffle=True,num_workers=2)
val_loader=torch.utils.data.DataLoader(val_set,batch_size=64,shuffle=True,num_workers=2)
test_loader=torch.utils.data.DataLoader(test_set,batch_size=64,shuffle=True,num_workers=2)

print(len(test_loader.dataset.classes))  # Number of classes
print(test_loader.dataset.classes)  # List of classes

10
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [24]:
class EarlyStop:
    def __init__(self, patience=10):
        # Number of epochs to wait without improvement before stopping
        self.patience = patience
        # Counter for epochs without improvement
        self.steps = 0
        # Initialize the minimum validation loss to infinity
        self.min_loss = float('inf')

    def stop(self, val_loss):
        # Check if the current validation loss is lower than the previous minimum
        if val_loss < self.min_loss:
            # Update the minimum validation loss
            self.min_loss = val_loss
            # Reset the counter since improvement was seen
            self.steps = 0
        else:
            # Increment the counter as no improvement was observed
            self.steps += 1
        
        # If the number of non-improving epochs reaches patience, stop training
        if self.steps >= self.patience:
            return True  # Stop training
        else:
            return False  # Continue training

# Example usage of EarlyStopping:
stoper = EarlyStop(patience=5)

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = nn.Sequential(
    nn.Linear(28*28,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,10).to(device)
)
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn=nn.CrossEntropyLoss()  
#In fact, the softmax layer is also inside CrossEntropyLoss, and we will not write it again

In [67]:
def train_epoch():
    tloss=0
    for imgs,labels in  train_loader:
        imgs= imgs.reshape(-1,28*28).to(device)
        labels=labels.reshape(-1,).to(device)
        preds=model(imgs)
        loss=loss_fn(preds,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tloss+=loss.detach()
    return tloss

def val_epoch():
    vloss=0
    for imgs,labels in val_loader:
        imgs= imgs.reshape(-1,28*28).to(device)
        labels=labels.reshape(-1,).to(device)
        preds=model(imgs)
        loss=loss_fn(preds,labels)
        vloss+=loss.detach()
    return vloss

In [55]:
#Using the functions we defined, we do the training
for i in range (1,10):
    tloss=train_epoch()
    vloss = val_epoch()
    print(f"at epoch {i} , tlossis {tloss} , vloss is {vloss}")

at epoch 1 , tlossis 196.17288208007812 , vloss is 35.85649108886719
at epoch 2 , tlossis 186.2237091064453 , vloss is 38.472206115722656
at epoch 3 , tlossis 179.30381774902344 , vloss is 39.436431884765625
at epoch 4 , tlossis 170.35552978515625 , vloss is 40.2406120300293
at epoch 5 , tlossis 161.9225616455078 , vloss is 39.65324401855469
at epoch 6 , tlossis 156.2237091064453 , vloss is 40.34807586669922
at epoch 7 , tlossis 148.79942321777344 , vloss is 38.720001220703125
at epoch 8 , tlossis 140.66334533691406 , vloss is 43.64981460571289
at epoch 9 , tlossis 135.5573272705078 , vloss is 43.89677810668945


In [34]:
#To get the prediction values, imgs and labels inputs should be one-dimensional
for i in range(20):
    img,labels=test_set[i]
    img=img.reshape(-1,28*28).to(device)
    pred=model(img)
    index_pred=torch.argmax(pred,dim=1) #argmax causes the index to give us the largest value
    idx=index_pred.item()
    print(f"{labels} the prediction is {idx}")

9 the prediction is 9
2 the prediction is 2
1 the prediction is 1
1 the prediction is 1
6 the prediction is 6
1 the prediction is 1
4 the prediction is 4
6 the prediction is 6
5 the prediction is 5
7 the prediction is 7
4 the prediction is 4
5 the prediction is 5
7 the prediction is 5
3 the prediction is 3
4 the prediction is 4
1 the prediction is 1
2 the prediction is 2
4 the prediction is 2
8 the prediction is 8
0 the prediction is 0


In [71]:
result= []
for imgs,labels in test_loader:
    imgs= imgs.reshape(-1,28*28).to(device)
    labels= (labels).reshape(-1,).to(device)
    preds=model(imgs)
    #print(preds)
    pred10=torch.argmax(preds,dim=1)
    #print(pred10) 
    correct =(pred10==labels) #True or False
    result.append(correct.detach().cpu().numpy().mean())
#print(result)

accuracy = np.array(result).mean()
print(accuracy)

0.8785828025477707


### Accuracy Result

The model achieved an **accuracy** of:

**`0.8786`** 

This value represents the average accuracy calculated over all batches in the test dataset.
